# Collecting Data Notebook
This is a notebook aimed at collecting data from the Pro-Football-Reference website

In [379]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from urllib.request import urlopen
from bs4 import BeautifulSoup

import webbrowser as wb

pd.options.display.max_colwidth = 100

## Uselul commands
Do not run unless initialized

In [174]:
url_wr_draft_1 = 'https://www.pro-football-reference.com/play-index/draft-finder.cgi?request=1&year_min=1936&year_max=2019&type=&round_min=1&round_max=30&slot_min=1&slot_max=500&league_id=&team_id=&pos%5B%5D=WR&college_id=all&conference=any&show=all&offset=0'
html =urlopen(url_wr_draft_1)
soup = BeautifulSoup(html,'html.parser')

wb.open_new_tab(url_wr_draft_1)
table_0 = soup.find_all('table')[0]
rowx= table_0.find_all('tr')[4]
columnx = rowx.find_all('td')[3]
linkx = columnx.find_all('a')[0]
print('https://www.pro-football-reference.com'+linkx.get('href'))

#list of added columns
['cfb '+ x for x in table2.columns.tolist()] + ['nfl '+ x for x in table3.columns.tolist()]

https://www.pro-football-reference.com/players/S/SamuDe00.htm


## Long Lists

In [380]:
list_of_draft = ['year','round','pick','player','NFL-url','pos','draft age','team',
                                  'entry year','last year','1st team pro select','pro select','weighted career AV',
                                  'years as primary starter','games', 'games started',
                                  'rushing attempts','rushing yards','rushing TD',
                                  'receiving attemps','receiving yards','receiving td',
                                  'college','link']

list_of_college = ['school','conference','class','pos','games',
                                  'receptions','yards', 'average','td',
                                 'attemps rushing','yards rushing','avg rushing','td rushing',
                                 'scrimmages','yards total','avg total','td total']

list_of_nfl = ['age','team','pos','no','game','game started',
                                 'target','receptions','yards','y/r','td','first downs','longest rec',
                                 'rec per game','yards per game','catch ratio','yards per target',
                                 'rushes','rush yards','rush td','first downs rush','longest rush',
                                 'rush yards per attempt','rush yards per game','rush attempt per games',
                                 'total touches','yards per touch','yards from scrimmage','total td','fumbles','AV']

list_of_added_columns = ['cfb school', 'cfb conference', 'cfb class', 'cfb pos', 'cfb games', 'cfb receptions', 'cfb yards',
 'cfb average', 'cfb td', 'cfb attemps rushing', 'cfb yards rushing', 'cfb avg rushing', 'cfb td rushing', 'cfb scrimmages', 'cfb yards total',
 'cfb avg total', 'cfb td total', 'nfl age', 'nfl team', 'nfl pos', 'nfl no', 'nfl game', 'nfl game started', 'nfl target', 'nfl receptions',
 'nfl yards', 'nfl y/r', 'nfl td', 'nfl first downs', 'nfl longest rec', 'nfl rec per game', 'nfl yards per game', 'nfl catch ratio',
 'nfl yards per target', 'nfl rushes', 'nfl rush yards', 'nfl rush td', 'nfl first downs rush', 'nfl longest rush', 'nfl rush yards per attempt',
 'nfl rush yards per game', 'nfl rush attempt per games', 'nfl total touches', 'nfl yards per touch', 'nfl yards from scrimmage', 'nfl total td',
 'nfl fumbles', 'nfl AV']

# Functions

In [381]:
def tabledraft(soup):

    table_0 = soup.find_all('table')[0]

    table = pd.DataFrame(columns= list_of_draft
                         ,index=range(0,310)) # I know the size 

    row_marker = 0
    for row in table_0.find_all('tr')[2:]:
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            if column_marker == 3:
                table.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
                table.iat[row_marker,column_marker] = 'https://www.pro-football-reference.com'+column.find_all('a')[0].get('href')
                column_marker += 1
            elif column_marker == 23:
                lista = column.find_all('a')
                if len(lista) == 0:
                    table.iat[row_marker,column_marker] = 'link is missing' 
                else:
                    table.iat[row_marker,column_marker] = lista[0].get('href') 
                    column_marker += 1
            else:
                table.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
        row_marker += 1

    table.dropna(subset=['year'],inplace=True)
    table.reset_index(drop=True)

    return table.reset_index(drop = True)

def url_fill_in(table):
    list_of_missing_index = table.index[table.loc[:,'link'] == 'link is missing'].tolist()
    table_edit = table

    for i in list_of_missing_index:
        x = table.loc[i,'player'].lower().split(' ')
        table_edit.loc[i,'link'] = 'https://www.sports-reference.com/cfb/players/'+x[0]+'-'+x[1]+'-1.html'
    
    return table_edit

def expend_draft_table_columns(table):
    table[list_of_added_columns] = pd.DataFrame([ [""] * 48], index=table.index)
    
    return table
    

def collegestats(soup):
    table_0 = soup.find_all('table')[0]
    table = pd.DataFrame(columns=list_of_college,
                         index=range(0,15)) # I know the size 
    
    row_marker = 0
    for row in table_0.find_all('tr')[2:]:
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            table.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        row_marker += 1
    
    table.dropna(subset=['school'],inplace=True)
    table.reset_index(drop=True)    


    return table

def row_cfb(t):
    return t.iloc[-2][0:4].tolist()+ [str(t['games'].apply(pd.to_numeric).iloc[:-1].sum())] + t.iloc[-1][5:].tolist()



def nflstats(soup):
    table_0 = soup.find_all('table')[1]
    table = pd.DataFrame(columns=list_of_nfl,
                         index=range(0,15)) # I know the size 
    
    row_marker = 0
    for row in table_0.find_all('tr')[2:]:
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            table.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        row_marker += 1
    
    table.dropna(subset=['team'],inplace=True)
    table.reset_index(drop=True)
    
    lr=table.iloc[-1]
    lrp= shiftrow(lr)
    
    table.drop(table.tail(1).index,inplace=True)
    table.loc[len(table)]=lrp

    return table

def shiftrow(row):
    return ['','','','']+row.values.tolist()[3:-1]

def row_nfl(t):
    return t.iloc[-2][0:4].tolist()+ t.iloc[-1][4:].tolist()

In [410]:
def main(url_draft):
    soup_draft = BeautifulSoup(urlopen(url_draft),'html.parser')
    table_draft0 = tabledraft(soup_draft)
    table_draft1 = url_fill_in(table_draft0)
    table_draft = expend_draft_table_columns(table_draft1)
        
    
    for i in table_draft.index[:3]:
        url_cfb = table_draft.loc[i]['link']
        soup_cfb = BeautifulSoup(urlopen(url_cfb),'html.parser')
        table_cfb = collegestats(soup_cfb)
        
        url_nfl = table_draft.loc[i]['NFL-url']
        soup_nfl = BeautifulSoup(urlopen(url_nfl),'html.parser')
        table_nfl = nflstats(soup_nfl)
        
        table_draft.loc[i,24:72] = row_cfb(table_cfb) + row_nfl(table_nfl)
        
    return table_draft

In [411]:
x = main(url_wr_draft_1)

### Code

In [383]:
url_wr_draft_1 = 'https://www.pro-football-reference.com/play-index/draft-finder.cgi?request=1&year_min=1936&year_max=2019&type=&round_min=1&round_max=30&slot_min=1&slot_max=500&league_id=&team_id=&pos%5B%5D=WR&college_id=all&conference=any&show=all&offset=0'
html =urlopen(url_wr_draft_1)
soup = BeautifulSoup(html,'html.parser')

table0=tabledraft(soup)

In [384]:
table1 = url_fill_in(table0)

table1

#table1 = expend_draft_table_columns(table1)

,year,round,pick,player,NFL-url,pos,draft age,team,entry year,last year,...,games,games started,rushing attempts,rushing yards,rushing TD,receiving attemps,receiving yards,receiving td,college,link
0,2019,1,25,Marquise Brown,https://www.pro-football-reference.com/players/B/BrowMa04.htm,WR,22,BAL,2019,2019,...,14,11,,,,46,584,7,Oklahoma,https://www.sports-reference.com/cfb/players/marquise-brown-1.html
1,2019,1,32,N'Keal Harry,https://www.pro-football-reference.com/players/H/HarrNK00.htm,WR,21,NWE,2019,2019,...,7,5,5,49,0,12,105,2,Arizona St.,https://www.sports-reference.com/cfb/players/nkeal-harry-1.html
2,2019,2,36,Deebo Samuel,https://www.pro-football-reference.com/players/S/SamuDe00.htm,WR,23,SFO,2019,2019,...,15,11,14,159,3,57,802,3,South Carolina,https://www.sports-reference.com/cfb/players/deebo-samuel-1.html
3,2019,2,51,A.J. Brown,https://www.pro-football-reference.com/players/B/BrowAJ00.htm,WR,22,TEN,2019,2019,...,16,11,3,60,1,52,1051,8,Mississippi,https://www.sports-reference.com/cfb/players/aj-brown-3.html
4,2019,2,56,Mecole Hardman,https://www.pro-football-reference.com/players/H/HardMe00.htm,WR,21,KAN,2019,2019,...,16,5,4,17,0,26,538,6,Georgia,https://www.sports-reference.com/cfb/players/mecole-hardman-1.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2010,4,99,Mardy Gilyard,https://www.pro-football-reference.com/players/G/GilyMa00.htm,WR,23,STL,2010,2012,...,19,2,,,,8,78,0,Cincinnati,https://www.sports-reference.com/cfb/players/mardy-gilyard-1.html
296,2010,4,101,Mike Williams,https://www.pro-football-reference.com/players/W/WillMi04.htm,WR,23,TAM,2010,2014,...,63,52,1,3,0,223,3089,26,Syracuse,https://www.sports-reference.com/cfb/players/mike-williams-4.html
297,2010,4,107,Marcus Easley,https://www.pro-football-reference.com/players/E/EaslMa00.htm,WR,22,BUF,2012,2015,...,42,0,,,,3,71,1,Connecticut,https://www.sports-reference.com/cfb/players/marcus-easley-1.html
298,2010,4,108,Jacoby Ford,https://www.pro-football-reference.com/players/F/FordJa01.htm,WR,23,OAK,2010,2013,...,38,13,19,201,2,57,848,3,Clemson,https://www.sports-reference.com/cfb/players/jacoby-ford-1.html


### Trying things out with Julio...

In [386]:
#index 255
julio_jones_college_url = table1.loc[table1.loc[:,'player'] == 'Julio Jones'].link.item()
print(julio_jones_college_url)
julio_jones_nfl_url = table1.loc[table1.loc[:,'player'] == 'Julio Jones']['NFL-url'].item()
print(julio_jones_nfl_url)

https://www.sports-reference.com/cfb/players/julio-jones-1.html
https://www.pro-football-reference.com/players/J/JoneJu02.htm


/Users/adriensaremi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/Users/adriensaremi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: `item` has been deprecated and will be removed in a future version
  after removing the cwd from sys.path.


In [387]:
url_cfb_sample = julio_jones_college_url
html =urlopen(url_cfb_sample)
soup = BeautifulSoup(html,'html.parser')

table2=collegestats(soup)

row_cfb(table2)

['Alabama',
 'SEC',
 'JR',
 'WR',
 '40.0',
 '179',
 '2653',
 '14.8',
 '15',
 '10',
 '139',
 '13.9',
 '2',
 '189',
 '2792',
 '14.8',
 '17']

In [388]:
url_nfl_sample = julio_jones_nfl_url
html =urlopen(url_nfl_sample)
soup = BeautifulSoup(html,'html.parser')

table3=nflstats(soup)

row_nfl(table3)

['30',
 'ATL',
 'WR',
 '11',
 '126',
 '125',
 '1252',
 '797',
 '12125',
 '15.2',
 '57',
 '574',
 '81',
 '6.3',
 '96.2',
 '63.7%',
 '9.7',
 '19',
 '118',
 '0',
 '11',
 '19',
 '6.2',
 '0.9',
 '0.2',
 '816',
 '15.0',
 '12243',
 '57',
 '11',
 '113']

In [390]:
#table2.iloc[-2][0:4].tolist()+ [str(table2['games'].apply(pd.to_numeric).iloc[:-1].sum())] + table2.iloc[-1][5:].tolist()
#table3.iloc[-2][0:4].tolist()+ table3.iloc[-1][4:].tolist()


table1.loc[255,24:72] = table2.iloc[-2][0:4].tolist()+ [str(table2['games'].apply(pd.to_numeric).iloc[:-1].sum())] + table2.iloc[-1][5:].tolist() + table3.iloc[-2][0:4].tolist()+ table3.iloc[-1][4:].tolist()

table1.loc[250:258]

,year,round,pick,player,NFL-url,pos,draft age,team,entry year,last year,...,nfl longest rush,nfl rush yards per attempt,nfl rush yards per game,nfl rush attempt per games,nfl total touches,nfl yards per touch,nfl yards from scrimmage,nfl total td,nfl fumbles,nfl AV
250,2012,7,231,Toney Clemons,https://www.pro-football-reference.com/players/C/ClemTo00.htm,WR,23,PIT,2012,2012,...,,,,,,,,,,
251,2012,7,235,Jeremy Ebert,https://www.pro-football-reference.com/players/E/EberJe00.htm,WR,23,NWE,2012,2013,...,,,,,,,,,,
252,2012,7,238,Junior Hemingway,https://www.pro-football-reference.com/players/H/HemiJu00.htm,WR,23,KAN,2012,2014,...,,,,,,,,,,
253,2012,7,244,Jordan White,https://www.pro-football-reference.com/players/W/WhitJo02.htm,WR,24,NYJ,2012,2012,...,,,,,,,,,,
254,2011,1,4,A.J. Green,https://www.pro-football-reference.com/players/G/GreeA.00.htm,WR,23,CIN,2011,2018,...,,,,,,,,,,
255,2011,1,6,Julio Jones,https://www.pro-football-reference.com/players/J/JoneJu02.htm,WR,22,ATL,2011,2019,...,19,6.2,0.9,0.2,816,15.0,12243,57,11,113
256,2011,1,26,Jonathan Baldwin,https://www.pro-football-reference.com/players/B/BaldJo00.htm,WR,22,KAN,2011,2013,...,,,,,,,,,,
257,2011,2,44,Titus Young,https://www.pro-football-reference.com/players/Y/YounTi00.htm,WR,22,DET,2011,2012,...,,,,,,,,,,
258,2011,2,58,Torrey Smith,https://www.pro-football-reference.com/players/S/SmitTo02.htm,WR,22,BAL,2011,2018,...,,,,,,,,,,
